In [1]:
filepath = "dspg_hotline/" #need path
bucketname = "callaudiofiles_dspg"

# Link to Google_Documentation(Allows you to reproduce the code below): https://cloud.google.com/speech-to-text/docs/python-client-migration


In [2]:
#Importing libraries 

from pydub import AudioSegment

import io

import os

#from google.cloud import speech
#from google.cloud.speech import enums
#from google.cloud.speech import types

from google.cloud import speech_v1p1beta1 as speech #Changed
from google.cloud.speech_v1p1beta1 import enums #Changed
from google.cloud.speech_v1p1beta1 import types #Changed


import wave

from google.cloud import storage

os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="Your Api Key" 

/home/am8gy/.local/lib/python3.7/site-packages/pydub/utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [5]:
def upload_blob(bucket_name, source_file_name, destination_blob_name):
    """Uploads a file to the bucket."""
    storage_client = storage.Client()
    bucket = storage_client.get_bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    blob.upload_from_filename(source_file_name)


In [6]:
def delete_blob(bucket_name, blob_name):
    """Deletes a blob from the bucket."""
    storage_client = storage.Client()
    bucket = storage_client.get_bucket(bucket_name)
    blob = bucket.blob(blob_name)

    blob.delete()

In [7]:
def speech_to_text_api(wav_file):
    
    file_name = filepath + wav_file
    
    bucket_name = bucketname
    source_file_name = filepath + wav_file
    destination_blob_name = wav_file
    
    upload_blob(bucket_name, source_file_name, destination_blob_name)
    
    gcs_uri = 'gs://' + bucketname + '/' + wav_file  # The URI of the audio file being transcribed.
    transcript = ''
    
    client = speech.SpeechClient() # Creates the Client
    audio = types.RecognitionAudio(uri=gcs_uri) 
                                                                    # identify audio content from a local file
    config = types.RecognitionConfig(                             # identify  from a Google Cloud Storage URI, use the new RecognitionAudio
    encoding=enums.RecognitionConfig.AudioEncoding.LINEAR16, #Encoding
    language_code='en-US',
    enable_speaker_diarization=True,  #enables speaker detection
    diarization_speaker_count=2) #specifies the estimated number of speakers in the conversation

    
    operation = client.long_running_recognize(config, audio) # Performs asynchronous speech recognition: receive results via the google.longrunning.Operations interface
    response = operation.result(timeout=10000) 
    result = response.results[-1] 
    words = result.alternatives[0].words
    
    tag=1 
    speaker="" 

    for word_information in words: 
        if word_information.speaker_tag==tag: 
            speaker=speaker+" "+word_information.word 
        else: 
            transcript += "speaker {}: {}".format(tag,speaker) + '\n' 
            tag=word_information.speaker_tag 
            speaker=""+word_information.word 
 
    transcript += "speaker {}: {}".format(tag,speaker) 
    
    delete_blob(bucket_name, destination_blob_name)
    return transcript

In [8]:
t_1 = speech_to_text_api("6183.wav")
t_2 = speech_to_text_api("6130.wav") 
t_3 = speech_to_text_api("4902.wav")
t_4 = speech_to_text_api("4790.wav")
t_5 = speech_to_text_api("4875.wav")

In [9]:
import pandas as pd

In [10]:
df = pd.DataFrame({"Database": "Talk_Bank", "Study": "Call_Home", "Call_Number": "6183", "File_Format": ".wav", 'Call_Transcript': [t_1]})
df2 = pd.DataFrame({"Database": "Talk_Bank", "Study": "Call_Home", "Call_Number": "6130", "File_Format": ".wav", 'Call_Transcript': [t_2]})
df3 = pd.DataFrame({"Database": "Talk_Bank", "Study": "Call_Home", "Call_Number": "4902", "File_Format": ".wav", 'Call_Transcript': [t_3]})
df4 = pd.DataFrame({"Database": "Talk_Bank", "Study": "Call_Home", "Call_Number": "4790", "File_Format": ".wav", 'Call_Transcript': [t_4]})
df5 = pd.DataFrame({"Database": "Talk_Bank", "Study": "Call_Home", "Call_Number": "4875", "File_Format": ".wav", 'Call_Transcript': [t_5]})

In [11]:

df_final_rough = pd.concat([df, df2, df3, df4, df5])

In [12]:
df_final_rough

,Database,Study,Call_Number,File_Format,Call_Transcript
0,Talk_Bank,Call_Home,6183,.wav,speaker 1: \nspeaker 2: are you there\nspeaker...
0,Talk_Bank,Call_Home,6130,.wav,speaker 1: \nspeaker 2: hello\nspeaker 1: hell...
0,Talk_Bank,Call_Home,4902,.wav,speaker 1: I\nspeaker 2: don't know if I don'...
0,Talk_Bank,Call_Home,4790,.wav,speaker 1: \nspeaker 2: are you there you got ...
0,Talk_Bank,Call_Home,4875,.wav,speaker 1: Hi how are you good to\nspeaker 2:...


In [ ]:
df_final_rough.drop(df_final_rough.columns[[0]], axis=1, inplace=True)

In [ ]:
df_final_rough.to_csv("rough_data_trans.csv")